In [9]:
import pathlib
import pickle
from PIL import Image
import io
from uuid import uuid4

from azure.storage.blob import ContainerClient
import matplotlib.pyplot as plt
import numpy as np
from joblib import Parallel, delayed

from experiment_utils.secrets import Secrets
from experiment_utils.utils import getLogger
from experiment_utils.constants import *

from csgo_clips_autotrim.feature_extraction import DownsampleConfig, get_downsampled_frames

In [2]:
logger = getLogger(__name__)

In [3]:
secrets = Secrets.load()

In [5]:
container_client = ContainerClient.from_container_url(container_url=secrets.AZURE_BLOBSTORE_CONTAINER_URL, credential=secrets.AZURE_BLOBSTORE_SAS_TOKEN)

In [11]:
def get_png_image_data(img_arr):
    data = io.BytesIO()
    plt.imsave(data, img_arr, cmap='gray')
    plt.imsave(DATA_DIR / 'test.png', img_arr, cmap='gray')
    return data.getvalue()

In [18]:
def upload_task(file_path: pathlib.Path, output_prefix: str):
    '''
    Upload frames from a given video to blob store.
    '''
    log = getLogger('upload_task')
    log.info('Uploading training data from video: %s', file_path)
    video_file_name, extension = file_path.name.split('.mp4')
    frame_name_prefix = f'{video_file_name}'
    frames = get_downsampled_frames(file_path.absolute().as_posix(), DownsampleConfig.from_str("downsample_1920x1080_60_RGB"))
    for idx, frame in enumerate(frames):
        data = get_png_image_data(frame)
        frame_file_name = f'{frame_name_prefix}_{idx:06}.png'
        container_client.upload_blob(f'{output_prefix}/{frame_file_name}', data, overwrite=True)

In [19]:
def upload_training_images(input_folder: pathlib.Path, output_prefix: str):
    '''
    Upload frames from the clips in the input folder to blob storage.
    '''
    tasks = []
    
    for path in input_folder.glob("*.mp4"):
        if not path.is_file():
            continue
        
        tasks.append(delayed(upload_task)(path, output_prefix))
    
    tic = time.perf_counter()
    Parallel(n_jobs=4)(tasks)
    toc = time.perf_counter()
    
    logger.info("Upload took: %dms", (toc - tic) * 1000)

In [20]:
upload_training_images(DATA_DIR / 'clips' / 'input', 'label-studio/src/frames')

[INFO] 2022-10-23 15:20:27,716 upload_task : Uploading training data from video: /code/data/clips/input/blue_and_ash_fail.mp4
[INFO] 2022-10-23 15:20:27,716 upload_task : Uploading training data from video: /code/data/clips/input/ashtm_kit_useless.mp4
[INFO] 2022-10-23 15:20:27,731 upload_task : Uploading training data from video: /code/data/clips/input/ashtm_tactical_nade.mp4
  0%|          | 0/3599 [00:00<?, ?it/s][INFO] 2022-10-23 15:20:27,754 upload_task : Uploading training data from video: /code/data/clips/input/B_rush_stopped.mp4
3600it [00:26, 137.03it/s] [00:26<00:00, 137.01it/s]
100%|██████████| 3600/3600 [00:20<00:00, 177.20it/s]
3600it [00:20, 177.57it/s]                          
100%|██████████| 3600/3600 [00:19<00:00, 181.46it/s]
[INFO] 2022-10-23 15:36:35,212 __main__    : Upload took: 968921ms
